In [1]:
# !psql --help

In [2]:
%env PGINSTANCE=pg

env: PGINSTANCE=pg


In [3]:
%env BASE_DIR=/home/jupyter/pagila

env: BASE_DIR=/home/jupyter/pagila


Create the pagila schema and objects by running pagila-schema.sql.

In [4]:
!psql -c "\i $BASE_DIR/pagila-schema.sql" --quiet

psql:/home/jupyter/pagila/pagila-schema.sql:8: NOTICE:  drop cascades to 47 other objects
DETAIL:  drop cascades to type pagila.mpaa_rating
drop cascades to type pagila.year
drop cascades to function pagila._group_concat(text,text)
drop cascades to function pagila.film_in_stock(integer,integer)
drop cascades to function pagila.film_not_in_stock(integer,integer)
drop cascades to function pagila.get_customer_balance(integer,timestamp with time zone)
drop cascades to function pagila.inventory_held_by_customer(integer)
drop cascades to function pagila.inventory_in_stock(integer)
drop cascades to function pagila.last_day(timestamp with time zone)
drop cascades to function pagila.last_updated()
drop cascades to sequence pagila.customer_customer_id_seq
drop cascades to table pagila.customer
drop cascades to function pagila.rewards_report(integer,numeric)
drop cascades to function pagila.group_concat(text)
drop cascades to sequence pagila.actor_actor_id_seq
drop cascades to table pagila.actor


 Populate the pagila tables by running pagila-insert-data.sql. 

In [5]:
!psql -c "\i $BASE_DIR/pagila-insert-data.sql" --quiet

 setval 
--------
    200
(1 row)

 setval 
--------
    605
(1 row)

 setval 
--------
     16
(1 row)

 setval 
--------
    600
(1 row)

 setval 
--------
    109
(1 row)

 setval 
--------
    599
(1 row)

 setval 
--------
   1000
(1 row)

 setval 
--------
   4581
(1 row)

 setval 
--------
      6
(1 row)

 setval 
--------
  32098
(1 row)

 setval 
--------
  16049
(1 row)

 setval 
--------
      2
(1 row)

 setval 
--------
      2
(1 row)



Describe each table by running "\d {table_name}" in psql. 

In [6]:
!psql -c "\d pagila.customer"

                                                Table "pagila.customer"
   Column    |           Type           | Collation | Nullable |                        Default                        
-------------+--------------------------+-----------+----------+-------------------------------------------------------
 customer_id | integer                  |           | not null | nextval('pagila.customer_customer_id_seq1'::regclass)
 store_id    | smallint                 |           | not null | 
 first_name  | text                     |           | not null | 
 last_name   | text                     |           | not null | 
 email       | text                     |           |          | 
 address_id  | smallint                 |           | not null | 
 activebool  | boolean                  |           | not null | true
 create_date | date                     |           | not null | 'now'::text::date
 last_update | timestamp with time zone |           |          | now()
 active      | 

In [7]:
!psql -c "\d pagila.actor"

                                              Table "pagila.actor"
   Column    |           Type           | Collation | Nullable |                    Default                     
-------------+--------------------------+-----------+----------+------------------------------------------------
 actor_id    | integer                  |           | not null | nextval('pagila.actor_actor_id_seq'::regclass)
 first_name  | text                     |           | not null | 
 last_name   | text                     |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "actor_pkey" PRIMARY KEY, btree (actor_id)
    "idx_actor_last_name" btree (last_name)
Referenced by:
    TABLE "pagila.film_actor" CONSTRAINT "film_actor_actor_id_fkey" FOREIGN KEY (actor_id) REFERENCES pagila.actor(actor_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON pagila.actor FOR EACH ROW EXECUTE FUNCTION pagila.last_updated()



In [8]:
!psql -c "\d pagila.category"

                                               Table "pagila.category"
   Column    |           Type           | Collation | Nullable |                       Default                        
-------------+--------------------------+-----------+----------+------------------------------------------------------
 category_id | integer                  |           | not null | nextval('pagila.category_category_id_seq'::regclass)
 name        | text                     |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "category_pkey" PRIMARY KEY, btree (category_id)
Referenced by:
    TABLE "pagila.film_category" CONSTRAINT "film_category_category_id_fkey" FOREIGN KEY (category_id) REFERENCES pagila.category(category_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON pagila.category FOR EACH ROW EXECUTE FUNCTION pagila.last_updated()



In [9]:
!psql -c "\d pagila.film"

                                                  Table "pagila.film"
        Column        |           Type           | Collation | Nullable |                   Default                    
----------------------+--------------------------+-----------+----------+----------------------------------------------
 film_id              | integer                  |           | not null | nextval('pagila.film_film_id_seq'::regclass)
 title                | text                     |           | not null | 
 description          | text                     |           |          | 
 release_year         | pagila.year              |           |          | 
 language_id          | smallint                 |           | not null | 
 original_language_id | smallint                 |           |          | 
 rental_duration      | smallint                 |           | not null | 3
 rental_rate          | numeric(4,2)             |           | not null | 4.99
 length               | smallint         

In [10]:
!psql -c "\d pagila.film_actor"

                        Table "pagila.film_actor"
   Column    |           Type           | Collation | Nullable | Default 
-------------+--------------------------+-----------+----------+---------
 actor_id    | smallint                 |           | not null | 
 film_id     | smallint                 |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "film_actor_pkey" PRIMARY KEY, btree (actor_id, film_id)
    "idx_fk_film_id" btree (film_id)
Foreign-key constraints:
    "film_actor_actor_id_fkey" FOREIGN KEY (actor_id) REFERENCES pagila.actor(actor_id) ON UPDATE CASCADE ON DELETE RESTRICT
    "film_actor_film_id_fkey" FOREIGN KEY (film_id) REFERENCES pagila.film(film_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON pagila.film_actor FOR EACH ROW EXECUTE FUNCTION pagila.last_updated()



In [11]:
!psql -c "\d pagila.film_category"

                      Table "pagila.film_category"
   Column    |           Type           | Collation | Nullable | Default 
-------------+--------------------------+-----------+----------+---------
 film_id     | smallint                 |           | not null | 
 category_id | smallint                 |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "film_category_pkey" PRIMARY KEY, btree (film_id, category_id)
Foreign-key constraints:
    "film_category_category_id_fkey" FOREIGN KEY (category_id) REFERENCES pagila.category(category_id) ON UPDATE CASCADE ON DELETE RESTRICT
    "film_category_film_id_fkey" FOREIGN KEY (film_id) REFERENCES pagila.film(film_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON pagila.film_category FOR EACH ROW EXECUTE FUNCTION pagila.last_updated()



In [12]:
!psql -c "\d pagila.address"

                                               Table "pagila.address"
   Column    |           Type           | Collation | Nullable |                      Default                       
-------------+--------------------------+-----------+----------+----------------------------------------------------
 address_id  | integer                  |           | not null | nextval('pagila.address_address_id_seq'::regclass)
 address     | text                     |           | not null | 
 address2    | text                     |           |          | 
 district    | text                     |           | not null | 
 city_id     | smallint                 |           | not null | 
 postal_code | text                     |           |          | 
 phone       | text                     |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "address_pkey" PRIMARY KEY, btree (address_id)
    "idx_fk_city_id" btree (city_id)
Foreign-key cons

In [13]:
!psql -c "\d pagila.city"

                                             Table "pagila.city"
   Column    |           Type           | Collation | Nullable |                   Default                    
-------------+--------------------------+-----------+----------+----------------------------------------------
 city_id     | integer                  |           | not null | nextval('pagila.city_city_id_seq'::regclass)
 city        | text                     |           | not null | 
 country_id  | smallint                 |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "city_pkey" PRIMARY KEY, btree (city_id)
    "idx_fk_country_id" btree (country_id)
Foreign-key constraints:
    "city_country_id_fkey" FOREIGN KEY (country_id) REFERENCES pagila.country(country_id) ON UPDATE CASCADE ON DELETE RESTRICT
Referenced by:
    TABLE "pagila.address" CONSTRAINT "address_city_id_fkey" FOREIGN KEY (city_id) REFERENCES pagila.city(city_id) ON UPDATE CASCADE ON 

In [14]:
!psql -c "\d pagila.country"

                                               Table "pagila.country"
   Column    |           Type           | Collation | Nullable |                      Default                       
-------------+--------------------------+-----------+----------+----------------------------------------------------
 country_id  | integer                  |           | not null | nextval('pagila.country_country_id_seq'::regclass)
 country     | text                     |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "country_pkey" PRIMARY KEY, btree (country_id)
Referenced by:
    TABLE "pagila.city" CONSTRAINT "city_country_id_fkey" FOREIGN KEY (country_id) REFERENCES pagila.country(country_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON pagila.country FOR EACH ROW EXECUTE FUNCTION pagila.last_updated()



In [15]:
!psql -c "\d pagila.inventory"

                                                Table "pagila.inventory"
    Column    |           Type           | Collation | Nullable |                        Default                         
--------------+--------------------------+-----------+----------+--------------------------------------------------------
 inventory_id | integer                  |           | not null | nextval('pagila.inventory_inventory_id_seq'::regclass)
 film_id      | smallint                 |           | not null | 
 store_id     | smallint                 |           | not null | 
 last_update  | timestamp with time zone |           | not null | now()
Indexes:
    "inventory_pkey" PRIMARY KEY, btree (inventory_id)
    "idx_store_id_film_id" btree (store_id, film_id)
Foreign-key constraints:
    "inventory_film_id_fkey" FOREIGN KEY (film_id) REFERENCES pagila.film(film_id) ON UPDATE CASCADE ON DELETE RESTRICT
    "inventory_store_id_fkey" FOREIGN KEY (store_id) REFERENCES pagila.store(store_id) ON UPDA

In [16]:
!psql -c "\d pagila.language"

                                               Table "pagila.language"
   Column    |           Type           | Collation | Nullable |                       Default                        
-------------+--------------------------+-----------+----------+------------------------------------------------------
 language_id | integer                  |           | not null | nextval('pagila.language_language_id_seq'::regclass)
 name        | character(20)            |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "language_pkey" PRIMARY KEY, btree (language_id)
Referenced by:
    TABLE "pagila.film" CONSTRAINT "film_language_id_fkey" FOREIGN KEY (language_id) REFERENCES pagila.language(language_id) ON UPDATE CASCADE ON DELETE RESTRICT
    TABLE "pagila.film" CONSTRAINT "film_original_language_id_fkey" FOREIGN KEY (original_language_id) REFERENCES pagila.language(language_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    las

In [17]:
!psql -c "\d pagila.payment"

                                               Table "pagila.payment"
    Column    |           Type           | Collation | Nullable |                      Default                       
--------------+--------------------------+-----------+----------+----------------------------------------------------
 payment_id   | integer                  |           | not null | nextval('pagila.payment_payment_id_seq'::regclass)
 customer_id  | smallint                 |           | not null | 
 staff_id     | smallint                 |           | not null | 
 rental_id    | integer                  |           | not null | 
 amount       | numeric(5,2)             |           | not null | 
 payment_date | timestamp with time zone |           | not null | 
Indexes:
    "payment_pkey" PRIMARY KEY, btree (payment_id)
    "idx_fk_customer_id" btree (customer_id)
    "idx_fk_staff_id" btree (staff_id)
Foreign-key constraints:
    "customer_id_fkey" FOREIGN KEY (customer_id) REFERENCES pagila.custom

In [18]:
!psql -c "\d pagila.rental"

                                               Table "pagila.rental"
    Column    |           Type           | Collation | Nullable |                     Default                      
--------------+--------------------------+-----------+----------+--------------------------------------------------
 rental_id    | integer                  |           | not null | nextval('pagila.rental_rental_id_seq'::regclass)
 rental_date  | timestamp with time zone |           | not null | 
 inventory_id | integer                  |           | not null | 
 customer_id  | smallint                 |           | not null | 
 return_date  | timestamp with time zone |           |          | 
 staff_id     | smallint                 |           | not null | 
 last_update  | timestamp with time zone |           | not null | now()
Indexes:
    "rental_pkey" PRIMARY KEY, btree (rental_id)
    "idx_fk_inventory_id" btree (inventory_id)
    "idx_unq_rental_rental_date_inventory_id_customer_id" UNIQUE, btree 

In [19]:
!psql -c "\d pagila.staff"

                                              Table "pagila.staff"
   Column    |           Type           | Collation | Nullable |                    Default                     
-------------+--------------------------+-----------+----------+------------------------------------------------
 staff_id    | integer                  |           | not null | nextval('pagila.staff_staff_id_seq'::regclass)
 first_name  | text                     |           | not null | 
 last_name   | text                     |           | not null | 
 address_id  | smallint                 |           | not null | 
 email       | text                     |           |          | 
 store_id    | smallint                 |           | not null | 
 active      | boolean                  |           | not null | true
 username    | text                     |           | not null | 
 password    | text                     |           |          | 
 last_update | timestamp with time zone |           | not null 

In [20]:
!psql -c "\d pagila.store"

                                                Table "pagila.store"
      Column      |           Type           | Collation | Nullable |                    Default                     
------------------+--------------------------+-----------+----------+------------------------------------------------
 store_id         | integer                  |           | not null | nextval('pagila.store_store_id_seq'::regclass)
 manager_staff_id | smallint                 |           | not null | 
 address_id       | smallint                 |           | not null | 
 last_update      | timestamp with time zone |           | not null | now()
Indexes:
    "store_pkey" PRIMARY KEY, btree (store_id)
    "idx_unq_manager_staff_id" UNIQUE, btree (manager_staff_id)
Foreign-key constraints:
    "store_address_id_fkey" FOREIGN KEY (address_id) REFERENCES pagila.address(address_id) ON UPDATE CASCADE ON DELETE RESTRICT
Referenced by:
    TABLE "pagila.customer" CONSTRAINT "customer_store_id_fkey" FOREIGN 

Write 3 queries that use an INNER JOIN between two or more tables. The queries
should also include a WHERE clause and ORDER BY clause. Feel free to include a
LIMIT clause if the output returns > 10 rows. Add a short Markdown comment in your
notebook above each SQL statement. 

1. Found all customer names with addresses in the system and their postal codes with last name starting with 'G', sorted by postal code.

In [92]:
!psql -c "SELECT first_name, last_name, postal_code FROM pagila.customer INNER JOIN pagila.address ON pagila.customer.address_id = pagila.address.address_id WHERE last_name LIKE 'G%' ORDER BY postal_code LIMIT 10;"

 first_name | last_name | postal_code 
------------+-----------+-------------
 NELLIE     | GARRETT   | 14408
 ALEX       | GRESHAM   | 16119
 RUBEN      | GEARY     | 16370
 HARVEY     | GUAJARDO  | 18043
 JOE        | GILLILAND | 18841
 BRANDY     | GRAVES    | 24645
 RANDY      | GAITHER   | 26385
 ADAM       | GOOCH     | 2738
 DENNIS     | GILMAN    | 31483
 JOANN      | GARDNER   | 3313
(10 rows)



2. Found all movies that had categories listed and their categories where the movie title started with 'G', sorted by film_id.

In [91]:
!psql -c "SELECT title, name FROM pagila.film INNER JOIN pagila.film_category ON pagila.film.film_id = pagila.film_category.film_id INNER JOIN pagila.category ON pagila.film_category.category_id = pagila.category.category_id WHERE title LIKE 'G%' ORDER BY film.film_id LIMIT 10;"

       title        |   name    
--------------------+-----------
 GABLES METROPOLIS  | Family
 GALAXY SWEETHEARTS | Classics
 GAMES BOWFINGER    | Travel
 GANDHI KWAI        | Family
 GANGS PRIDE        | Animation
 GARDEN ISLAND      | Sci-Fi
 GASLIGHT CRUSADE   | Horror
 GATHERING CALENDAR | Games
 GENTLEMEN STAGE    | Foreign
 GHOST GROUNDHOG    | Children
(10 rows)



3. Found all the cities and countries where the city starts with 'G', sorted by country name alphabetically.

In [90]:
!psql -c "SELECT city, country FROM pagila.city INNER JOIN pagila.country ON pagila.city.country_id = pagila.country.country_id WHERE city LIKE 'G%' ORDER BY country LIMIT 10;"

    city     |   country   
-------------+-------------
 Graz        | Austria
 Goinia      | Brazil
 Guaruj      | Brazil
 Gatineau    | Canada
 Gulbarga    | India
 Gandhinagar | India
 Gorontalo   | Indonesia
 Guadalajara | Mexico
 Gingoog     | Philippines
 Gijn        | Spain
(10 rows)



Write 3 queries that use an OUTER JOIN between two or more tables. The
queries should also include a WHERE clause and ORDER BY clause. Feel free to
include a LIMIT clause if the output returns > 10 rows. Again, add a short Markdown
comment above each SQL statement. 

1. Looking at all customers that had payments with amount more than $10.

In [89]:
!psql -c "SELECT first_name, last_name, amount FROM pagila.customer FULL OUTER JOIN pagila.payment ON pagila.customer.customer_id = pagila.payment.customer_id WHERE amount > 10 ORDER BY amount LIMIT 10;"

 first_name | last_name | amount 
------------+-----------+--------
 BRANDY     | GRAVES    |  10.99
 LONNIE     | TIRADO    |  10.99
 RITA       | GRAHAM    |  10.99
 DAVID      | ROYAL     |  10.99
 VIOLET     | RODRIQUEZ |  10.99
 DON        | BONE      |  10.99
 DAN        | PAINE     |  10.99
 CHESTER    | BENNER    |  10.99
 ELMER      | NOE       |  10.99
 CODY       | NOLEN     |  10.99
(10 rows)



2. Selected the employee name and store ID for any employees with last name starting with 'H' (there was only one).

In [88]:
!psql -c "SELECT first_name, last_name, pagila.store.store_id FROM pagila.store FULL OUTER JOIN pagila.staff ON pagila.store.store_id = pagila.staff.store_id WHERE last_name LIKE 'H%' ORDER BY last_name;"

 first_name | last_name | store_id 
------------+-----------+----------
 Mike       | Hillyer   |        1
(1 row)



3. Found movies and their film_ids for movies that are ininventory at store 1, sorted by film_id.

In [87]:
!psql -c "SELECT DISTINCT title, pagila.film.film_id FROM pagila.film FULL OUTER JOIN pagila.inventory ON pagila.film.film_id = pagila.inventory.film_id WHERE store_id = 1 ORDER BY pagila.film.film_id LIMIT 10;"

      title       | film_id 
------------------+---------
 ACADEMY DINOSAUR |       1
 AFFAIR PREJUDICE |       4
 AGENT TRUMAN     |       6
 AIRPLANE SIERRA  |       7
 ALABAMA DEVIL    |       9
 ALADDIN CALENDAR |      10
 ALAMO VIDEOTAPE  |      11
 ALASKA PHANTOM   |      12
 ALIEN CENTER     |      15
 ALLEY EVOLUTION  |      16
(10 rows)



Write 1 query that uses a self join on any table. Feel free to include a LIMIT clause
if the output returns > 10 rows. Add a short Markdown comment above each SQL
statement.

Finding all the customers that have the same last name (there are none).

In [86]:
!psql -c "SELECT  c1.last_name, c1.first_name, c2.last_name, c2.first_name FROM pagila.customer AS c1 INNER JOIN pagila.customer AS c2 ON (c1.last_name = c2.last_name AND c1.customer_id != c2.customer_id);"

 last_name | first_name | last_name | first_name 
-----------+------------+-----------+------------
(0 rows)

